In [8]:
%pip install supabase

import requests
import pandas as pd

from io import StringIO

import supabase


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
base_url = "http://127.0.0.1:8000/"
market_data_endpoint = base_url + "market-data/"

In [ ]:
resp = requests.get(market_data_endpoint + "historical?tickers=TSLA,AMZN&period=1d&interval=60m").json()

res = {}
for ticker, df_json in resp.items():
    res[ticker] = pd.read_json(
        StringIO(df_json),
        orient='index',
        convert_dates=True
    )

res

{'TSLA':                                 Close        High         Low        Open  \
 2025-08-13 13:30:00+00:00  345.089905  348.980011  340.239990  341.880005   
 2025-08-13 14:30:00+00:00  340.785004  345.562408  338.549988  345.100006   
 2025-08-13 15:30:00+00:00  341.300110  342.329987  340.200012  340.815002   
 2025-08-13 16:30:00+00:00  341.505005  341.659912  339.250000  341.299988   
 2025-08-13 17:30:00+00:00  340.225006  341.819794  338.230011  341.505005   
 2025-08-13 18:30:00+00:00  340.720001  341.454987  339.200012  340.260010   
 2025-08-13 19:30:00+00:00  339.339996  341.250000  339.200012  340.760010   
 
                              Volume  
 2025-08-13 13:30:00+00:00  23273591  
 2025-08-13 14:30:00+00:00  12224375  
 2025-08-13 15:30:00+00:00   6451030  
 2025-08-13 16:30:00+00:00   6283472  
 2025-08-13 17:30:00+00:00   6884780  
 2025-08-13 18:30:00+00:00   5159534  
 2025-08-13 19:30:00+00:00   4064061  ,
 'AMZN':                                 Close       

In [49]:
resp = requests.get(market_data_endpoint + "prices?tickers=TSLA,AMZN").json()
resp

{'TSLA': {'Datetime': '2025-08-13T15:59:00-04:00',
  'Open': 339.75,
  'High': 339.80999755859375,
  'Low': 339.20001220703125,
  'Close': 339.3399963378906,
  'Volume': 713716,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 'AMZN': {'Datetime': '2025-08-13T15:59:00-04:00',
  'Open': 224.6699981689453,
  'High': 224.6699981689453,
  'Low': 224.49000549316406,
  'Close': 224.55499267578125,
  'Volume': 751311,
  'Dividends': 0.0,
  'Stock Splits': 0.0}}

In [8]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
from supabase import create_client, Client
from dotenv import load_dotenv


In [22]:
# test_portfolios.py
import requests
import os
import json


# --------------------
# CONFIG
# --------------------
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")  # or anon key if email/password login
BASE_URL = "http://127.0.0.1:8000"

EMAIL = "test@example.com"
PASSWORD = "password123"

# --------------------
# 1. Get Supabase Access Token
# --------------------
auth_res = requests.post(
    f"{SUPABASE_URL}/auth/v1/token?grant_type=password",
    headers={
        "apikey": SUPABASE_KEY,
        "Content-Type": "application/json"
    },
    json={
        "email": EMAIL,
        "password": PASSWORD
    }
)

if auth_res.status_code != 200:
    print("❌ Failed to log in:", auth_res.status_code, auth_res.text)
    exit(1)

access_token = auth_res.json()["access_token"]
print(f"✅ Logged in as {EMAIL}")
print(f"Access Token: {access_token[:20]}...")

# --------------------
# 2. Call /portfolios
# --------------------
headers = {
    "Authorization": f"Bearer {access_token}"
}

portfolios_res = requests.get(f"{BASE_URL}/portfolios", headers=headers)

print("\nStatus:", portfolios_res.status_code)
try:
    print(json.dumps(portfolios_res.json(), indent=4))
except Exception:
    print(portfolios_res.text)


✅ Logged in as test@example.com
Access Token: eyJhbGciOiJIUzI1NiIs...

Status: 200
[
    {
        "id": "591dd7f5-fff1-4041-ae74-3507688da719",
        "created_at": "2025-08-14T17:08:27.123472+00:00",
        "name": "Test Portfolio",
        "last_updated": "2025-08-14T17:08:27.123472+00:00",
        "initial_investment": 5000.0,
        "capital": 0.0,
        "user_id": "d62c7b0a-4af1-4520-9f35-ec7825d8c227",
        "order_history": [
            {
                "id": 1,
                "timestamp": "2025-08-14T17:12:49.375759+00:00",
                "symbol": "AAPL",
                "quantity": 10,
                "price": 145.23,
                "portfolio_id": "591dd7f5-fff1-4041-ae74-3507688da719"
            },
            {
                "id": 2,
                "timestamp": "2025-08-14T17:12:49.375759+00:00",
                "symbol": "TSLA",
                "quantity": 5,
                "price": 625.1,
                "portfolio_id": "591dd7f5-fff1-4041-ae74-3507688d

In [ ]:
import requests
import os
import json

# ======== CONFIG ========
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")  # or anon key if using public route
API_BASE_URL = "http://127.0.0.1:8000"

EMAIL = "test@example.com"
PASSWORD = "password123"

# ======== STEP 1: LOGIN TO GET TOKEN ========
login_res = requests.post(
    f"{SUPABASE_URL}/auth/v1/token?grant_type=password",
    headers={
        "apikey": SUPABASE_KEY,
        "Content-Type": "application/json"
    },
    json={
        "email": EMAIL,
        "password": PASSWORD
    }
)

if login_res.status_code != 200:
    print("❌ Failed to log in:", login_res.text)
    exit()

token = login_res.json()["access_token"]
print("✅ Got token:", token[:20] + "...")

# ======== STEP 2: CREATE PORTFOLIO ========

create_res = requests.post(
    f"{API_BASE_URL}/portfolios",
    headers={
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    },
    json={
        "name": "Create Test Portfolio",
        "initial_investment": 10000.00,
    }
)
print("\n📌 Response Status:", create_res.status_code)
print("📌 Response Body:\n", json.dumps(create_res.json(), indent=4))


✅ Got token: eyJhbGciOiJIUzI1NiIs...

📌 Response Status: 200
📌 Response Body:
 {
    "id": "57867417-1da2-4bf9-8129-f707898f34c2",
    "created_at": "2025-08-14T18:04:11.449186+00:00",
    "name": "Create Test Portfolio",
    "last_updated": "2025-08-14T18:04:11.449186+00:00",
    "initial_investment": 10000.0,
    "capital": 10000.0,
    "user_id": "d62c7b0a-4af1-4520-9f35-ec7825d8c227"
}


In [34]:
import requests
import os
import json

# --------------------
# CONFIG
# --------------------
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")
BASE_URL = "http://127.0.0.1:8000"

EMAIL = "test@example.com"
PASSWORD = "password123"

# Portfolio UUID you want to delete
PORTFOLIO_ID = "57867417-1da2-4bf9-8129-f707898f34c2"

# --------------------
# 1. Get Supabase Access Token
# --------------------
auth_res = requests.post(
    f"{SUPABASE_URL}/auth/v1/token?grant_type=password",
    headers={
        "apikey": SUPABASE_KEY,
        "Content-Type": "application/json"
    },
    json={
        "email": EMAIL,
        "password": PASSWORD
    }
)

if auth_res.status_code != 200:
    print("❌ Failed to log in:", auth_res.status_code, auth_res.text)
    exit(1)

access_token = auth_res.json()["access_token"]
print(f"✅ Logged in as {EMAIL}")
print(f"Access Token: {access_token[:20]}...")

# --------------------
# 2. Delete Portfolio
# --------------------
headers = {
    "Authorization": f"Bearer {access_token}"
}

delete_res = requests.delete(f"{BASE_URL}/portfolios/{PORTFOLIO_ID}", headers=headers)

print("\nStatus:", delete_res.status_code)
try:
    print(json.dumps(delete_res.json(), indent=4))
except Exception:
    print(delete_res.text)


✅ Logged in as test@example.com
Access Token: eyJhbGciOiJIUzI1NiIs...

Status: 200
{
    "message": "Portfolio 57867417-1da2-4bf9-8129-f707898f34c2 deleted successfully"
}


In [29]:
# test_create_order.py
import requests
import os
from dotenv import load_dotenv

import json
load_dotenv()
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")
BASE_URL = "http://127.0.0.1:8000"

EMAIL = "test@example.com"
PASSWORD = "password123"

PORTFOLIO_ID = "591dd7f5-fff1-4041-ae74-3507688da719"

# 1. Get token
auth_res = requests.post(
    f"{SUPABASE_URL}/auth/v1/token?grant_type=password",
    headers={
        "apikey": SUPABASE_KEY,
        "Content-Type": "application/json"
    },
    json={
        "email": EMAIL,
        "password": PASSWORD
    }
)
access_token = auth_res.json()["access_token"]

headers = {
    "Authorization": f"Bearer {access_token}"
} 

# 2. Create BUY order (+10 quantity)
order_res = requests.post(
    f"{BASE_URL}/portfolios/{PORTFOLIO_ID}/orders",
    headers=headers,
    json={
        "symbol": "AMZN",
        "quantity": -10,
        "price": 175.50
    }
)

print("BUY Status:", order_res.status_code)
print(json.dumps(order_res.json(), indent=4))



BUY Status: 200
{
    "status": "success",
    "order": {
        "timestamp": "2025-08-15T19:17:31.910148+00:00",
        "symbol": "AMZN",
        "quantity": -10,
        "price": 175.5,
        "portfolio_id": "591dd7f5-fff1-4041-ae74-3507688da719",
        "order_id": "bcd43548-263a-4398-93a7-dbb384f16caa"
    }
}
